In [1]:
from simtk import openmm
from simtk.openmm import app, unit
import mdtraj as md
import parmed
from copy import deepcopy
import numpy as np

# Compare amber vs omm energies using amber topology for both

In [27]:
# Load prmtop and inpcrd files
parm_amber = parmed.load_file("../files/glycam/rbd_ace2/RBD_ACE2_complex_FullGlycos.prmtop", "../files/glycam/rbd_ace2/RBD_ACE2_complex_FullGlycos.inpcrd")


In [10]:
# Load the ffxml
ffxml = ['../ffxml/protein.ff14SB.xml', "../ffxml/GLYCAM_06j-1.xml", "../ffxml/tip3p_standard.xml"]
ff = app.ForceField(*ffxml)

In [41]:
# Create a new topology with H-H bonds in water removed
source_topology = parm_amber.topology
destination_topology = app.Topology()

new_atoms = {}
for chain in source_topology.chains():
    new_chain = destination_topology.addChain(chain.id)
    for residue in chain.residues():
        new_residue = destination_topology.addResidue(residue.name, new_chain, residue.id)
        for atom in residue.atoms():
            new_atom = destination_topology.addAtom(atom.name, atom.element, new_residue, atom.id)
            new_atoms[atom] = new_atom
for bond in source_topology.bonds():
    if bond[0].name in ['H1', 'H2'] and bond[1].name in ['H1', 'H2']:
        continue
    order = bond.order
    destination_topology.addBond(new_atoms[bond[0]], new_atoms[bond[1]], order=order)

In [ ]:
system_amber = parm_amber.createSystem()

In [42]:
# Create OpenMM system
system_omm = ff.createSystem(destination_topology)

In [52]:
# set beta
temperature = 300.0 * unit.kelvin
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
kT = kB * temperature
beta = 1.0/kT

def compute_potential_components(system, positions, beta=beta):
    # Note: this is copied from perses
    # TODO: consider moving this outside of assert_energies2()
    system = deepcopy(system)
    for index in range(system.getNumForces()):
        force = system.getForce(index)
        force.setForceGroup(index)
    integrator = openmm.VerletIntegrator(1.0*unit.femtosecond)
    platform = openmm.Platform.getPlatformByName('Reference')
    context = openmm.Context(system, integrator, platform)
    context.setPositions(positions)
    energy_components = list()
    for index in range(system.getNumForces()):
        force = system.getForce(index)
        forcename = force.__class__.__name__
        groups = 1<<index
        potential = beta * context.getState(getEnergy=True, groups=groups).getPotentialEnergy()
        energy_components.append((forcename, potential))
    del context, integrator
    return energy_components

In [53]:
amber_energies = compute_potential_components(system_amber, parm_amber.positions)
omm_energies = compute_potential_components(system_omm, parm_amber.positions)
for amber, omm in zip(amber_energies, omm_energies):
    force_name = amber[0]
    print(force_name, amber[1], omm[1])

HarmonicBondForce 1705.7637802995876 1697.7917138842663
HarmonicAngleForce 4319.87498025964 4319.861478533262
PeriodicTorsionForce 17558.831087421866 17558.873616264365
NonbondedForce -1526621.0087427753 -1526621.0093642396
CMMotionRemover 0.0 0.0


In [ ]:
# The bonds are off here because I got rid of the H1-H2 bonds in waters

# Try parametrizing RBD:ACE2 all in openmm

In [3]:
# rbd_pdb = app.PDBFile("../files/glycam/rbd_ace2/3_refined_ace2_rbd_complex_RBD_ONLY_cleaned.pdb")
# ace2_pdb = app.PDBFile("../files/glycam/rbd_ace2/3_refined_ace2_rbd_complex_ACE2_ONLY_cleaned.pdb")
# rbd_md_topology = md.Topology.from_openmm(rbd_pdb.topology)
# ace2_md_topology = md.Topology.from_openmm(ace2_pdb.topology)
# complex_md_topology = rbd_md_topology.join(ace2_md_topology)
# complex_topology = complex_md_topology.to_openmm()
# complex_positions = unit.Quantity(np.zeros([rbd_md_topology.n_atoms + ace2_md_topology.n_atoms, 3]), unit=unit.nanometers)
# complex_positions[:rbd_md_topology.n_atoms, :] = rbd_pdb.positions
# complex_positions[rbd_md_topology.n_atoms:, :] = ace2_pdb.positions

rbd_ace2_pdb = app.PDBFile("/home/zhangi/choderalab/openmmforcefields/amber/files/glycam/rbd_ace2/rbd_ace2_for_openmm_corrected_2.pdb")


In [4]:
# Load the ffxml
ffxml = ['/home/zhangi/choderalab/openmmforcefields/amber/ffxml/protein.ff14SB.xml', "/home/zhangi/choderalab/openmmforcefields/amber/ffxml/GLYCAM_06j-1.xml", "/home/zhangi/choderalab/openmmforcefields/amber/ffxml/tip3p_standard.xml"]
ff = app.ForceField(*ffxml)

In [5]:
modeller = app.Modeller(rbd_ace2_pdb.topology, rbd_ace2_pdb.positions)
modeller.loadHydrogenDefinitions("/home/zhangi/choderalab/openmmforcefields/amber/glycam/hydrogens_formatted.xml")
modeller.addHydrogens(ff)
modeller.addSolvent(ff, model='tip3p', padding=0.9 * unit.nanometers, ionicStrength=0.15 * unit.molar)

In [6]:
system_omm = ff.createSystem(modeller.topology)

# Check that the bonds are present connecting protein residues to glycans (they aren't showing up in PyMOL)

In [5]:
for bond in parm_amber.topology.bonds():
    if bond[0].residue.name == 'NLN' or bond[1].residue.name == 'NLN':
        print(bond)

Bond(<Atom 157 (CA) of chain 0 residue 11 (NLN)>, <Atom 155 (N) of chain 0 residue 11 (NLN)>)
Bond(<Atom 159 (CB) of chain 0 residue 11 (NLN)>, <Atom 157 (CA) of chain 0 residue 11 (NLN)>)
Bond(<Atom 162 (CG) of chain 0 residue 11 (NLN)>, <Atom 159 (CB) of chain 0 residue 11 (NLN)>)
Bond(<Atom 163 (OD1) of chain 0 residue 11 (NLN)>, <Atom 162 (CG) of chain 0 residue 11 (NLN)>)
Bond(<Atom 164 (ND2) of chain 0 residue 11 (NLN)>, <Atom 162 (CG) of chain 0 residue 11 (NLN)>)
Bond(<Atom 164 (ND2) of chain 0 residue 11 (NLN)>, <Atom 3001 (C1) of chain 0 residue 196 (UYB)>)
Bond(<Atom 166 (C) of chain 0 residue 11 (NLN)>, <Atom 157 (CA) of chain 0 residue 11 (NLN)>)
Bond(<Atom 166 (C) of chain 0 residue 11 (NLN)>, <Atom 168 (N) of chain 0 residue 12 (ALA)>)
Bond(<Atom 167 (O) of chain 0 residue 11 (NLN)>, <Atom 166 (C) of chain 0 residue 11 (NLN)>)
Bond(<Atom 153 (C) of chain 0 residue 10 (PHE)>, <Atom 155 (N) of chain 0 residue 11 (NLN)>)
Bond(<Atom 3778 (C) of chain 0 residue 240 (THR)>, <A